In [2]:
from collections import OrderedDict

class LastUpdatedOrderedDict(OrderedDict):
    """Store items in the order they were last updated"""

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)

In [3]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [4]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [5]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [6]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [7]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict(a='foo')
ad['a']

42

In [8]:
d = {}
d.update(ad) # ignored by __getitem__
d['a']

'foo'

In [9]:
d

{'a': 'foo'}

In [11]:
# subclassing dict, list is error prone, UserDict
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [12]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [13]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [14]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict2(a='foo')
ad['a']

42

In [15]:
d = {}
d.update(ad)
d['a']

42

In [16]:
d

{'a': 42}

In [20]:
class Root:
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'

class A(Root):
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()

class B(Root):
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')

class Leaf(A, B):
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [21]:
leaf1 = Leaf()
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [22]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [23]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

In [24]:
class U():
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()

class LeafUA(U, A):
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

In [25]:
u = U()
u.ping() # throw (no super class)

<__main__.U object at 0x112d77610>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [26]:
leaf2 = LeafUA()
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [27]:
LeafUA.__mro__

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

In [32]:
import collections

def _upper(key):
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

In [33]:
class UpperDict(UpperCaseMixin, collections.UserDict):
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):
    """Specialized 'counter' that uppercases string keys"""

In [34]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
list(d.keys())

['A', 2]

In [35]:
d['b'] = 'letter B'
'b' in d

True

In [36]:
d['a'], d.get('B')

('letter A', 'letter B')

In [37]:
d

{'A': 'letter A', 2: 'digit two', 'B': 'letter B'}

In [38]:
c = UpperCounter('BaNanA')
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]